# **Detect \#china hate Hate-speech-CNERG/bert-base-uncased-hatexplain**

reference: https://github.com/hate-alert/HateXplain

In [40]:
# check which gpu we're using
!nvidia-smi

Fri Mar 18 16:52:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    27W /  70W |   2398MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [41]:
!pip install simpletransformers

In [42]:
!pip install emoji

In [43]:
# # helper function to save predictions to an output file
# def labels2file(p, outf_path):
# 	with open(outf_path,'w') as outf:
# 		for pi in p:
# 			outf.write(','.join([str(k) for k in pi])+'\n')

In [44]:
# source: https://github.com/pysentimiento/pysentimiento/blob/master/pysentimiento/preprocessing.py

import emoji
import re

extra_args = {
    "vinai/bertweet-base":  {
        "user_token": "@USER",
        "url_token": "HTTPURL",
    }
}



special_tokens = ["@usuario", "url", "hashtag", "emoji"]


user_regex = re.compile(r"@[a-zA-Z0-9_]{0,15}")
url_regex = re.compile(
    r"((?<=[^a-zA-Z0-9])(?:https?\:\/\/|[a-zA-Z0-9]{1,}\.{1}|\b)(?:\w{1,}\.{1}){1,5}(?:com|co|org|edu|gov|uk|net|ca|de|jp|fr|au|us|ru|ch|it|nl|se|no|es|mil|iq|io|ac|ly|sm){1}(?:\/[a-zA-Z0-9]{1,})*)"
)

hashtag_regex = re.compile(r'\B#(\w*[a-zA-Z]+\w*)')
start_of_camel = re.compile(r'([A-Z]+)')

def camel_to_human(s, lower=True):
    """
    Converts camel case to 'human' case
    Arguments:
    ----------
    lower: bool (default: False)
        Convert output to lower
    """

    ret = start_of_camel.sub(r' \1', s).strip()

    if lower:
        ret = ret.lower()

    return ret

emoji_regex = re.compile(r"\|([^\|]+)\|")

def convert_emoji_to_text(x, emoji_wrapper="[EMOJI]"):
    """
    """
    return f" {emoji_wrapper} " + " ".join(x.groups()[0].split("_")) + f" {emoji_wrapper} "


replacements = {
    "~": None,
    "£": None,
    "¥": None,
    "¨": '"',
    "©": None,
    "«": '"',
    "°": None,
    "´": None,
    "¶": None,
    "·": None,
    "º": None,
    "»": '"',
    "×": None,
    "ا": None,
    "–": None,
    "—": None,
    "‘": None,
    "’": None,
    "“": '"',
    "”": '"',
    "•": None,
    "…": None,
    "‼": "!",
    "€": "$",
    "™": None,
    "●": None,
    "☠": None,
    "☹": None,
    "☺": None,
    "☻": "😃",
    "♀": None,
    "♂": None,
    "♡": "❤️",
    "♥": "❤️",
    "⚰": None,
    "⛱": None,
    "⛹": None,
    "✈": None,
    "✓": None,
    "❤": None,
    "ー": None,
    "🕯": None,
    "🛰": None,
}

laughter_conf = {
    "es": {
        "regex": re.compile("[ja][ja]+aj[ja]+"),
        "replacement": "jaja",
    },
    "en": {
        "regex": re.compile("[ha][ha]+ah[ha]+"),
        "replacement": "haha",
    }
}



def preprocess_tweet(
    text, lang="es", user_token="@usuario", url_token="url", preprocess_hashtags=True, hashtag_token=None,
    demoji=True, shorten=3, normalize_laughter=True, emoji_wrapper="emoji"):
    """
    Basic preprocessing
    Arguments:
    ---------
    text: str
        Text to preprocess
    lang: str (default 'es')
        Language used in the preprocessing. This is used for the demoji functionality and laughter preprocessing
    user_token: str (default "[USER]")
        Token used to replace user handles
    url_token: str (default "[URL]")
        Token used to replace urls
    preprocess_hashtags: boolean (default True)
        If true, applies preprocessing to hashtag, trying to split camel cases
    hashtag_token: str (default None)
        If preprocess_hashtags is True, adds hashtag_token before the preprocessed content of the hashtag
    shorten: int (default: 3)
        If not none, all occurrences of shorten or more characters are cut to this number
    demoji: boolean (default True)
        If true, converts emoji to text representations using `emoji` library, and wraps this with "[EMOJI]" strings
    normalize_laughter: boolean (default True)
        Normalizes laughters. Uses different regular expressions depending on the lang argument.
    """
    if lang == "en" and user_token == "@usuario":
        """
        If it is english and we didn't set any defaults, we set the vinai/bertweet-base defaults
        """
        user_token = "@USER"
        url_token = "HTTPURL"


    ret = ""
    for char in text:
        if char in replacements:
            replacement = replacements[char]
            if replacement:
                ret += replacement
        else:
            ret += char
    text = ret

    text = user_regex.sub(user_token, text)
    text = url_regex.sub(url_token, text)

    if shorten:
        repeated_regex = re.compile(r"(.)"+ r"\1" * (shorten-1) + "+")
        text = repeated_regex.sub(r"\1"*shorten, text)

    if demoji:
        text = emoji.demojize(text, language=lang, delimiters=("|", "|"))
        text = emoji_regex.sub(
            lambda x: convert_emoji_to_text(x, emoji_wrapper=emoji_wrapper),
            text
        )

    if normalize_laughter:
        laughter_regex = laughter_conf[lang]["regex"]
        replacement = laughter_conf[lang]["replacement"]

        text = laughter_regex.sub(
            replacement,
            text
        )

    def process_hashtags(x):
        """
        Hashtag preprocessing function
        Take first group and decamelize
        """


        text = x.groups()[0]

        text = camel_to_human(text)

        if hashtag_token:
            text = hashtag_token + " " + text

        return text

    if preprocess_hashtags:
        text = hashtag_regex.sub(
            process_hashtags,
            text
        )

    return text.strip()

## Load Data

In [45]:
import pandas as pd
from collections import Counter

In [46]:
df = pd.read_csv('2021_Q4_1.csv', lineterminator='\n')
len(df)

241244

In [47]:
df

,Unnamed: 0,author id,created_at,id,lang,like_count,quote_count,reply_count,retweet_count,source,tweet,CARDIFFNLP_ROBERTA_PREDS
0,0,49919826,2021-12-31 23:58:03+00:00,1477066552839004160,en,0,0,0,0,Instagram,Happy new year! \n#china #beijing #winter #eve...,0
1,1,38971530,2021-12-31 23:57:59+00:00,1477066538331086860,en,0,0,0,0,Paper.li,The latest The Political Science Daily! https:...,0
2,2,58010461,2021-12-31 23:57:37+00:00,1477066442705195008,en,0,0,0,0,Twitter Web App,Do you support this? Use your purchasing power...,0
3,3,1465236285526880260,2021-12-31 23:57:03+00:00,1477066301612957699,en,0,1,0,0,Twitter for Android,Thoughts?\n\n@beverleyturner @MaajidNawaz\n@Un...,0
4,4,783046681,2021-12-31 23:56:45+00:00,1477066228028133380,en,1,0,2,1,Twitter Web App,#Putin Tells #Biden &amp; #EU Cut #NATO Crap &...,0
...,...,...,...,...,...,...,...,...,...,...,...,...
241239,241239,758341474119520256,2021-10-01 00:00:26+00:00,1443727470197346309,en,0,0,0,0,WordPress.com,China's electric carmakers are darting into Eu...,0
241240,241240,50611261,2021-10-01 00:00:05+00:00,1443727380909023233,en,1,0,0,0,Oktopost,🇨🇳 News story - Argus' #copper concentrate TC/...,0
241241,241241,3508175477,2021-10-01 00:00:01+00:00,1443727366346485764,en,19,3,2,9,Twitter Web App,Today we mark the 72nd anniversary of the foun...,0
241242,241242,1219090523811528705,2021-10-01 00:00:00+00:00,1443727360621101072,en,13,4,1,21,Twitter Web App,🇨🇳🎉Today marks the 72nd anniversary of the fou...,0


In [48]:
tweets = df.tweet

## Preprocess Data

In [49]:
# preprocessing
preprocessed = []
for tweet in tweets:
  preprocessed_t = preprocess_tweet(tweet, lang='en')
  preprocessed.append(preprocessed_t)

In [50]:
preprocessed[:10]

['Happy new year! \nchina beijing winter evening #2021 #2022 sanlitun @USER Beijing HTTPURL',
 'The latest The Political Science Daily! HTTPURL Thanks to @USER cdnpoli china',
 'Do you support this? Use your purchasing power to vote! amazon amazon prime amazonギフト china hong kong democrat republican facebook google blue origin alexa kindle unlimited taiwan  taiwanese\nHTTPURL',
 "Thoughts?\n\n@USER @USER\n@USER @USER\n\nHow many of our politicians are working for/with china's ccp, or other groups that seek to dismantle democracy in the UK?\n\ncovid19 covidpassports\ncovid_19 vaccine passports\nlockdownuk\n\nHTTPURL",
 'putin Tells biden &amp; eu Cut nato Crap &amp; Get Real...The "Great Game" is OVER. HTTPURL ...msnbc foxnews nytimes cnn wsj cnbc politico huffpost newsmax oan gop npr fox nyt wapo nbc cbs  aarp biden pbs nyt china russia ukraine history g7',
 'A Digital Manhunt: How Chinese Police Track Critics on Twitter and Facebook \nchina surveillance state big brother no free speech

## Load Model

In [51]:
from simpletransformers.classification import ClassificationModel

model = ClassificationModel(
    "bert", "Hate-speech-CNERG/bert-base-uncased-hatexplain")

In [52]:
# get prreds and logits
preds, logits = model.predict(preprocessed)

  0%|          | 0/241244 [00:00<?, ?it/s]

  0%|          | 0/30156 [00:00<?, ?it/s]

In [53]:
Counter(preds)

Counter({0: 2473, 1: 236610, 2: 2161})

In [54]:
df['hateXplain_PREDS'] = preds
df.to_csv('2021_Q4_2.csv')

In [55]:
df

,Unnamed: 0,author id,created_at,id,lang,like_count,quote_count,reply_count,retweet_count,source,tweet,CARDIFFNLP_ROBERTA_PREDS,hateXplain_PREDS
0,0,49919826,2021-12-31 23:58:03+00:00,1477066552839004160,en,0,0,0,0,Instagram,Happy new year! \n#china #beijing #winter #eve...,0,1
1,1,38971530,2021-12-31 23:57:59+00:00,1477066538331086860,en,0,0,0,0,Paper.li,The latest The Political Science Daily! https:...,0,1
2,2,58010461,2021-12-31 23:57:37+00:00,1477066442705195008,en,0,0,0,0,Twitter Web App,Do you support this? Use your purchasing power...,0,1
3,3,1465236285526880260,2021-12-31 23:57:03+00:00,1477066301612957699,en,0,1,0,0,Twitter for Android,Thoughts?\n\n@beverleyturner @MaajidNawaz\n@Un...,0,1
4,4,783046681,2021-12-31 23:56:45+00:00,1477066228028133380,en,1,0,2,1,Twitter Web App,#Putin Tells #Biden &amp; #EU Cut #NATO Crap &...,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
241239,241239,758341474119520256,2021-10-01 00:00:26+00:00,1443727470197346309,en,0,0,0,0,WordPress.com,China's electric carmakers are darting into Eu...,0,1
241240,241240,50611261,2021-10-01 00:00:05+00:00,1443727380909023233,en,1,0,0,0,Oktopost,🇨🇳 News story - Argus' #copper concentrate TC/...,0,1
241241,241241,3508175477,2021-10-01 00:00:01+00:00,1443727366346485764,en,19,3,2,9,Twitter Web App,Today we mark the 72nd anniversary of the foun...,0,1
241242,241242,1219090523811528705,2021-10-01 00:00:00+00:00,1443727360621101072,en,13,4,1,21,Twitter Web App,🇨🇳🎉Today marks the 72nd anniversary of the fou...,0,1


In [56]:
# from google.colab import files
# files.download('2021-Q1-2.csv')